In [33]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import networkx as nx
import plotly.graph_objects as go
import ast
import requests as r
from io import StringIO

pd.set_option('display.max_columns', 50)

# Introdução

Em nossas análises, temos como meta principal o mapeamento dos jogos com maior taxa de engajamento.

Nesse sentido, após analisar as variáveis, decidimos por eliminar algumas delas:


'playerage' - Nos parece ter alguns problemas de formatação (problemas com a tabela ascii) e entrega um faixa de idade recomendada pelos jogadores. Consideramos muito subjetivo e prefirimos usar a 'minage', apesar de serem valores próximos na média;

'gamelink' - Se quisermos ver o jogo no BGG, basta pesquisá-lo por ser nome;

'boardgameexpansion' - Cada jogo tem a sua específica, não faz sentido analisar;

'max_community' (Fornecido pela comunidade) - No geral, igual ao 'maxplayers' (fornecido pelos fabricantes), só que com alguns valores faltando; Em geral, dados fornecidos pelo público podem conter problemas, como falta de votos em certo jogo, mas no geral são mais confiáveis.

'minplayers' - Fornecido pelos fabricantes, indicam o limite de jogabilidade, mas não garante uma experiência interessante. Sendo assim, vale usar a opinião da comunidade;

In [34]:
def faz_df(url):
    req = r.request('GET', url, headers={'user-agent': 'Mozilla/5.0'})
    df = pd.read_json(StringIO(req.text), encoding='UTF-8')
    return df

In [35]:
information_df = faz_df('https://treinamento-ndados.s3.sa-east-1.amazonaws.com/boardgames_information_raw.json')

In [36]:
df = pd.read_excel('df_unificado.xlsx')
df['minplayers'] = information_df['minplayers'].copy()

In [37]:
df.head()

,object_id,name,yearpublished,sortindex,maxplayers,minplaytime,maxplaytime,minage,min_community,totalvotes,languagedependence,usersrated,average,baverage,stddev,avgweight,numweights,numgeeklists,numtrading,numwanting,numcomments,siteviews,numplays,numplays_month,news,blogs,weblink,podcast,boardgamedesigner_cnt,boardgameartist_cnt,boardgamepublisher_cnt,boardgamehonor_cnt,boardgamecategory_cnt,boardgamemechanic_cnt,boardgameexpansion_cnt,boardgameversion_cnt,boardgamefamily_cnt,boardgamedesigner,boardgameartist,boardgamepublisher,boardgamehonor,boardgamecategory,boardgamemechanic,boardgamefamily,minplayers
0,174430,Gloomhaven,2017,1,4,60,120,12,3.0,827.0,4,31254.0,8.85292,8.58424,1.59819,3.8078,1311,3657,313,1365,5972,8933078,230213,3478,7,471,31,139,1,3,9,23,5,12,4,19,7,"['Isaac Childres""']","['Alexandr Elichev', 'Josh T. McDowell', 'Alva...","['Cephalofair Games', 'Albi', 'Asmodee', 'Feue...",['2017 Best Science Fiction or Fantasy Board G...,"['Adventure', 'Exploration', 'Fantasy', 'Fight...","['Campaign / Battle Card Driven', 'Cooperative...","['Campaign Games', 'Components: Miniatures', '...",1
1,161936,Pandemic Legacy Season 1,2015,2,4,60,60,13,4.0,549.0,4,34729.0,8.62499,8.47159,1.59463,2.8301,971,3612,272,771,5477,2971746,196621,1090,9,597,69,164,2,1,11,20,2,8,0,33,3,"['Rob Daviau""', 'Matt Leacock""']",['Chris Quilliams'],"['Z-Man Games', 'Asterion Press', 'Devir', 'Fi...",['2015 Cardboard Republic Immersionist Laurel ...,"['Environmental', 'Medical']","['Action Points', 'Cooperative Game', 'Hand Ma...","['Campaign Games', 'Legacy', 'Pandemic']",2
2,167791,Terraforming Mars,2016,3,5,120,120,12,3.0,1252.0,3,48339.0,8.42299,8.26781,1.36938,3.2313,1863,5941,277,2068,7274,4724387,245997,4380,14,1158,60,148,1,1,20,20,6,9,15,29,6,"['Jacob Fryxelius""']",['Isaac Fryxelius'],"['FryxGames', 'Arclight', 'Fantasmagoria', 'Gh...",['2016 Cardboard Republic Architect Laurel Nom...,"['Economic', 'Environmental', 'Industry / Manu...","['Card Drafting', 'End Game Bonuses', 'Hand Ma...","['Fryxgames Future Timeline', 'Planets: Mars',...",1
3,182028,Through the Ages A New Story of Civilization,2015,4,4,120,120,14,3.0,436.0,4,18269.0,8.49419,8.23513,1.49542,4.3850,891,2083,280,1049,2660,2448074,54284,511,5,185,30,42,1,4,13,3,3,5,1,14,2,"['Vlaada Chv\\u00e1til""']","['Filip Murmak', 'Radim Pech', 'Jakub Politzer...","['Czech Games Edition', 'Cranio Creations', 'D...",['2015 Golden Geek Best Strategy Board Game No...,"['Card Game', 'Civilization', 'Economic']","['Action Points', 'Auction/Bidding', 'Auction:...","['Tableau Building', 'Through the Ages']",2
4,224517,Brass Birmingham,2018,5,4,60,120,14,3.0,242.0,1,10070.0,8.62031,8.20459,1.22876,3.9122,467,1039,37,1077,1702,894621,23336,815,3,124,13,21,3,3,8,6,3,8,0,9,6,"['Gavan Brown""', 'Matt Tolman""', 'Martin Walla...","['Lina Cossette', 'David Forest', 'Damien Mamm...","['Roxley', 'BoardM Factory', 'Conclave Editora...",['2018 Golden Geek Best Board Game Artwork & P...,"['Economic', 'Industry / Manufacturing', 'Tran...","['Hand Management', 'Income', 'Loans', 'Market...","['Beer', 'Brass', 'Cities: Birmingham (England...",2


Após removermos as variáveis que eram explicitamente descartáveis, devemos remover as que contém dados aparentemente ruins e arrumar as que têm má formatação

In [38]:
# 'min_community' possui alguns valores nulos
min_df = df[['minplayers', 'min_community']]
min_df.dropna()
min_df.describe()

,minplayers,min_community
count,20016.000000,14149.000000
mean,2.055256,3.203336
std,0.745471,1.398786
min,0.000000,1.000000
25%,2.000000,2.000000
50%,2.000000,3.000000
75%,2.000000,4.000000
max,10.000000,31.000000


Vou preencher os valores nulos de 'min_community' com 'minplayers' + 2. Prefirimo os 'min_community' pela maior confiabilidade das opiniões dos avaliadores do BGG.

In [39]:
#decidimos por subistituir os valores nulos de 'min_community' por 'minplayers' + 2
df['min_community'].fillna(df['minplayers']+2, inplace=True)
df = df.drop("minplayers", axis=1)

Encontramos um problema: há diversos dados nulos nas colunas baverage e average, e não sabemos o que essa nulicidade representa diretamente. Portanto, vamos prever esses valores com base na razão entre average e baverage. A seguir fica o passo a passo de como fizemos o cálculo da razão.

Primeiramente, iremos tratar as linhas com nenhuma avaliação de jogadores. Ao fazer isso, notamos que essas linhas não contem dados releveantes em nenhuma outra coluna, como mostrado abaixo.

In [40]:
# Criando um novo DataFrame para manipulação
df_manipulado = df.copy()

Vou comparar os valores de cada variável que contém userrated == 0 com o valor total delas, para saber sua representatividade.

In [41]:
def verifica_porcentagem(df:pd.DataFrame, coluna):
    valor = df.query('usersrated == 0').sum()[coluna]
    total = df[coluna].sum()
    return valor / total * 100

In [42]:
colunas_analisadas = ['numweights','numgeeklists', 'numwanting', 'numcomments', 'siteviews', 'blogs', 'weblink']
for coluna in colunas_analisadas:
    print(f"A porcentagem de {coluna} é {verifica_porcentagem(df_manipulado, coluna)}")

A porcentagem de numweights é 0.00031196251458424755
A porcentagem de numgeeklists é 0.011253797901023462
A porcentagem de numwanting é 0.007868988949147294
A porcentagem de numcomments é 0.0009131586156515387
A porcentagem de siteviews é 0.02870729860782448
A porcentagem de blogs é 0.0004923682914820286
A porcentagem de weblink é 0.009585812996764788


Como todos os valores representam nem 1% do total, serão desconsiderados.

In [43]:
# Removendo todas as linhas com usersrated == 0
df_manipulado = df_manipulado.query('usersrated != 0').copy()

Analisando baverage == 0

In [44]:
print(f"A porcentagem de usersrated com baverage == 0 por total é {df_manipulado.query('baverage == 0')['usersrated'].sum() / df_manipulado['usersrated'].sum() * 100}")

A porcentagem de usersrated com baverage == 0 por total é 0.10620128769908506


Como as ocorrencias de baverage == 0 refletem 0.1% dos votos todais, optei por desonsiderá-las para efeito de cálculo

In [45]:
df_manipulado = df_manipulado.query('baverage != 0').copy()

Lidando com dados nulos

Como os dados nulos ocasionarão em futuros erros ou novos dados nulos após efetuação do cálculo, desconsideraremos

In [46]:
df_manipulado = df_manipulado.dropna().copy()

Cálculo da razão

In [47]:
df_manipulado['average/baverage'] = df_manipulado['average'] / df_manipulado['baverage']
df_manipulado['average/baverage_ponderado'] =df_manipulado['average/baverage'] * df_manipulado['usersrated']
razao_media = df_manipulado['average/baverage_ponderado'].sum() / df_manipulado['usersrated'].sum()

#### Remapeando o DataFrame original

 Removendo registros sem votos e tratando os dados nulos e zerados como o mesmo problema:

In [48]:
df = df.query('usersrated != 0').copy()
df = df.fillna(0)

 Removendo duplo 0 (average == 0 e baverage == 0)

In [49]:
df['average+baverage'] = df['average'] + df['baverage']
df = df.query('average+baverage != 0').copy()
df.drop(labels=['average+baverage'], axis=1, inplace=True)

Efetuando o mapeamento

In [50]:
# Defini duas funções para tratar os dados zerados(que antes eram nulos) das colunas
# average e baverage

def tratamento_average(linha):
    if linha['average'] <= 0.1:
        return linha['baverage'] * razao_media
    else:
        return linha['average']
    
def tratamento_baverage(linha):
    if linha['baverage'] <= 0.1:
        return linha['average'] / razao_media
    else:
        return linha['baverage']

In [51]:
# Atualizando a coluna average
df['average'] = df.apply(tratamento_average, axis=1)
# Atualizando a coluna baverage
df['baverage'] = df.apply(tratamento_baverage, axis=1)

A partir daqui, o nosso DF não possui mais nenhuma coluna com valores nulos ou desformatados. Sendo assim, devemos encontrar algum método de medir a popularidade de cada jogo, para assim descobrirmos quais categorias, mecanicas e familias de jogos são mais recorrentes nas vendas. Para isso, vamos utilizamos um método de machine learning chamado clusterização, o qual irá agrupar jogos similares em um mesmo 'cluster', com base em suas características como nota, total de avaliações, visitações em sua página no site, etc.

Com essa tecnica, portanto, poderemos juntar jogos mais populares e, posteriormente, analisar quais de suas características os tornam mais famosos.

## Machine Learning: clusterização dos dados estatísticos

Antes de começar, vamos selecionar as colunas que medem, estatísticamente, a popularidade/qualidade de um jogo. 

In [52]:
# Selecionei colunas que medem, estatísticamente, a popularidade de um jogo.
colunas_clusterizadas = ['usersrated', 'numgeeklists', 'numwanting', 'numcomments', 
                         'siteviews', 'news', 'blogs', 'weblink', 'podcast', 'average',
                          'baverage']
X = df[colunas_clusterizadas].copy()

##### Usarei o algoritmo de clusterização Kmeans

In [53]:
from sklearn.cluster import KMeans
from sklearn import metrics

Agrupei jogos em 5 grupos com base em suas estatísticas de popularidade. Assim, ao indentificaros os melhores grupos, poderemos analisar quais de suas características categóricas os fazem tão populares e, então, recomendar jogos com as mesmas características.

In [54]:
df_clusterizado = df[colunas_clusterizadas].copy()
model_stat = KMeans(n_clusters=5, random_state=0, n_init='auto')
labels_stat = model_stat.fit_predict(df_clusterizado)
df_clusterizado['Cluster'] = labels_stat

Descrição dos clusters:


In [55]:
# Função que agrupa o dataframe clusterizado e cria a coluna com a quantidade total de itens em cada cluster
def descricao(df: pd.DataFrame, grupamento: str) -> pd.DataFrame:
    descricao = df.groupby(grupamento)
    n = descricao.size()
    descricao = descricao.mean()
    descricao['Quantidade Total'] = n
    
    return descricao

In [56]:
descricao(df_clusterizado, 'Cluster')

,usersrated,numgeeklists,numwanting,numcomments,siteviews,news,blogs,weblink,podcast,average,baverage,Quantidade Total
Cluster,,,,,,,,,,,,
0,222.489819,108.769222,16.329698,76.928563,3.276307e+04,0.339769,3.768610,4.216090,0.701458,6.216062,5.553143,17974
1,10955.879365,2629.755556,480.533333,2351.187302,8.368732e+05,3.612698,130.158730,41.850794,24.307937,7.381665,7.011795,315
2,39694.172414,7448.620690,956.241379,7306.068966,3.858579e+06,8.896552,526.068966,94.206897,100.137931,7.864858,7.666282,29
3,2873.803818,1025.134957,166.907176,766.382488,2.793917e+05,1.615537,39.330481,18.776169,7.949967,7.022249,6.332230,1519
4,22617.531646,4172.139241,780.000000,4212.658228,1.745673e+06,5.278481,245.050633,62.379747,46.126582,7.610450,7.345284,79


Para indentificar qual cluster contém os melhores jogos, farei um gráfico comparando a nota média da BGG de cada cluster.

In [62]:
agrupamento_cluster = df_clusterizado.groupby(by=['Cluster']).mean()
fig_nota_cluster = px.bar(agrupamento_cluster, x=agrupamento_cluster.index, y='baverage', title='Nota média dos jogos em cada cluster')
fig_nota_cluster.show()

In [63]:
fig_avaliacoes_cluster = px.bar(agrupamento_cluster, x=agrupamento_cluster.index, y='usersrated', 
                                title='Quantidade de avaliações média de cada cluster')
fig_avaliacoes_cluster.show()

Com esses dois gráficos, podemos concluir que o cluster 2 e 4 são os mais bem avaliados e possuem a maior quantidade de avaliações. Nesse sentido, a partir de agora iremos avaliar qual as características dos jogos nesses clusters os fazem ser tão bem avaliados/famosos. 

Entretanto, vale ressaltar que os jogos do cluster 0, 1 e 3 não serão desconsiderados, eles serão usados posteriormente como sugestões que contém as mesmas características dos jogos dos clusters mais populares, mas que não atingem a qualidade/popularidade por conta de outros fatores, como um recente lançamento, por exemplo.

In [64]:
df['cluster'] = df_clusterizado['Cluster'].copy()

### Análise de categoria

Para a análise fazer sentido, vamos descartar todas as colunas que contem informações específicas de apenas um jogo, como quais e quantas expansões ele tem, e também dados como 'quais/quantos premios determinado jogo ganhou', visto que não dizem nada sobre o jogo em si. Além disso, descartamos categorias quantitativas irrelevantes como 'quantidade de mecânicas que o jogo tem' dado que o importante é qual mecânica o jogo possui, e não quantas.

Portanto, as características selecionadas foram: categorias, mecânicas e familias que os jogos pertencem.

É importante ressaltar, entretanto, que descartar essas informações agora não significa jogá-las no lixo. Posteriormente, elas podem ser usadas para o mapeamento dos jogos em grafos ou indentificação dos publicadores que mais lançam jogos com as categorias selecionadas aqui.

In [77]:
df_filtrado = df.drop(labels=['boardgamedesigner_cnt', 'boardgameartist_cnt', 'boardgamepublisher_cnt', 'boardgamehonor_cnt', 'boardgamecategory_cnt', 'boardgamemechanic_cnt', 'boardgameexpansion_cnt'
                      , 'boardgameversion_cnt', 'boardgamefamily_cnt', 'boardgamedesigner', 'boardgameartist', 'boardgamepublisher', 'boardgamehonor'
                      ], axis=1)

Primeiramente, vamos transformar os dados das categorias em listas.

In [78]:
df_filtrado['boardgamecategory'] = df_filtrado['boardgamecategory'].apply(ast.literal_eval)
df_filtrado['boardgamemechanic'] = df_filtrado['boardgamemechanic'].apply(ast.literal_eval)
df_filtrado['boardgamefamily'] = df_filtrado['boardgamefamily'].apply(ast.literal_eval)

Criarei dois dataframes agora, um apenas com o cluster 2 e outro com apenas o cluster 4.

In [81]:
df_filtrado_cluster2 = df_filtrado.query('cluster == 2').copy()
df_filtrado_cluster4 = df_filtrado.query('cluster == 4').copy()

Como cada jogo pode ter mais de uma mecanica e pertencer a mais de uma categoria/familia, defini uma função que conta quantas aparições cada valor unico de mecanica, categoria e familia aparecem. Com isso, poderemos analisar cada feature individualmente e extrair quais são mais populares.

In [82]:
# Função que retorna um data frame com a contagem que cada feature unico de um DataFrame
def conta_feature(df: pd.DataFrame, coluna) -> pd.DataFrame:
    count_df = {}
    for lista in df[coluna]:
        for valor in lista:
            count_df[valor] = count_df.get(valor, 0) + 1
    count_df = pd.DataFrame.from_dict(count_df, orient='index', columns=['contagem'])
    count_df.sort_values(by=['contagem'], ascending=False, inplace=True)
    count_df.reset_index(names='feature', inplace=True)
    return count_df